In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

url = "https://www.twse.com.tw/zh/page/trading/exchange/STOCK_DAY_AVG.html"
    
driver = webdriver.Chrome("/Users/hou-yehchen/Desktop/Python/spider/chromedriver")

driver.get(url)

In [4]:
s = driver.find_element_by_css_selector("#d1 > select:nth-child(2)")
Select(s).select_by_value("6")

In [5]:
stuck_input = driver.find_element_by_css_selector("#main-form > div > div > form > input")
stuck_input.send_keys("0050")

In [6]:
search = driver.find_element_by_css_selector("#main-form > div > div > form > a.button.search")
search.click()

In [7]:
from bs4 import BeautifulSoup as bs

In [18]:
soup = bs(driver.page_source, "html.parser")

for tr in soup.find("table").find("tbody").find_all("tr"):
    tds = tr.find_all("td")
    print("日期: "+tds[0].text, "收盤價: "+tds[1].text)

日期: 109/06/01 收盤價: 84.90
日期: 109/06/02 收盤價: 85.30
日期: 109/06/03 收盤價: 86.80
日期: 109/06/04 收盤價: 87.60
日期: 109/06/05 收盤價: 88.35
日期: 109/06/08 收盤價: 89.50
日期: 109/06/09 收盤價: 89.60
日期: 109/06/10 收盤價: 90.45
日期: 109/06/11 收盤價: 89.05
日期: 109/06/12 收盤價: 88.25
日期: 109/06/15 收盤價: 87.00
日期: 109/06/16 收盤價: 88.90
日期: 109/06/17 收盤價: 89.00
日期: 109/06/18 收盤價: 88.90
日期: 109/06/19 收盤價: 88.85
日期: 109/06/22 收盤價: 89.05
日期: 109/06/23 收盤價: 89.45
日期: 109/06/24 收盤價: 90.00
日期: 109/06/29 收盤價: 89.10
日期: 109/06/30 收盤價: 89.90
日期: 月平均收盤價 收盤價: 88.50
